In [2]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)
engine

Engine(sqlite:///:memory:)

In [3]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

### Mapped classes - Animal

In [4]:
from sqlalchemy.orm import Mapped, mapped_column
from sqlalchemy import String, ForeignKey
from typing import Optional

class Pessoa(Base):
    __tablename__ = 'PESSOA'

    id: Mapped[int] = mapped_column(primary_key=True)
    nome: Mapped[str] = mapped_column(String(50))
    sobrenome: Mapped[Optional[str]]

    def __repr__(self) -> str:
        return f'Pessoa(id={self.id!r}, nome={self.nome!r}, sobrenome={self.sobrenome!r})'

In [5]:
class Animal(Base):
    __tablename__ = 'ANIMAL'

    id: Mapped[int] = mapped_column(primary_key=True)
    nome: Mapped[str] = mapped_column(String(50))
    id_pessoa = mapped_column(ForeignKey('PESSOA.id'))

    def __repr__(self) -> str:
        return f'Animal(id={self.id!r}, nome={self.nome!r}, id_pessoa={self.id_pessoa!r})'

In [6]:
Base.metadata.create_all(engine)

2023-11-26 18:36:43,178 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-26 18:36:43,179 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PESSOA")
2023-11-26 18:36:43,179 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:36:43,180 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PESSOA")
2023-11-26 18:36:43,181 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:36:43,181 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ANIMAL")
2023-11-26 18:36:43,182 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:36:43,183 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ANIMAL")
2023-11-26 18:36:43,183 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:36:43,184 INFO sqlalchemy.engine.Engine 
CREATE TABLE "PESSOA" (
	id INTEGER NOT NULL, 
	nome VARCHAR(50) NOT NULL, 
	sobrenome VARCHAR, 
	PRIMARY KEY (id)
)


2023-11-26 18:36:43,184 INFO sqlalchemy.engine.Engine [no key 0.00052s] ()
2023-11-26 18:36:43,186 INFO sqlalchemy.engine.Engine 
CREATE TA

In [7]:
from sqlalchemy import inspect

inspect(engine).get_table_names()

2023-11-26 18:36:43,195 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-26 18:36:43,195 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-11-26 18:36:43,195 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:36:43,196 INFO sqlalchemy.engine.Engine ROLLBACK


['ANIMAL', 'PESSOA']

In [8]:
jubileu = Pessoa(nome='Jubileu')
jubileu

Pessoa(id=None, nome='Jubileu', sobrenome=None)

In [9]:
jose_pocai = Pessoa(nome='José', sobrenome='Pocai')
jose_pocai

Pessoa(id=None, nome='José', sobrenome='Pocai')

In [10]:
from sqlalchemy.orm import Session

session = Session(engine, expire_on_commit=False)
session

In [11]:
session.add_all([jubileu, jose_pocai])

In [12]:
session.new #stage

IdentitySet([Pessoa(id=None, nome='Jubileu', sobrenome=None), Pessoa(id=None, nome='José', sobrenome='Pocai')])

In [13]:
session.commit()

2023-11-26 18:36:43,235 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-26 18:36:43,237 INFO sqlalchemy.engine.Engine INSERT INTO "PESSOA" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-26 18:36:43,238 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Jubileu', None)
2023-11-26 18:36:43,239 INFO sqlalchemy.engine.Engine INSERT INTO "PESSOA" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-26 18:36:43,239 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('José', 'Pocai')
2023-11-26 18:36:43,240 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
print(jose_pocai)
jubileu

Pessoa(id=2, nome='José', sobrenome='Pocai')


Pessoa(id=1, nome='Jubileu', sobrenome=None)

In [15]:
session.close()

In [16]:
with Session(engine) as session:
    print(session.get(Pessoa, 1))

2023-11-26 18:36:43,258 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-26 18:36:43,260 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id AS "PESSOA_id", "PESSOA".nome AS "PESSOA_nome", "PESSOA".sobrenome AS "PESSOA_sobrenome" 
FROM "PESSOA" 
WHERE "PESSOA".id = ?
2023-11-26 18:36:43,261 INFO sqlalchemy.engine.Engine [generated in 0.00063s] (1,)
Pessoa(id=1, nome='Jubileu', sobrenome=None)
2023-11-26 18:36:43,262 INFO sqlalchemy.engine.Engine ROLLBACK


In [17]:
from sqlalchemy import select

print(select(Pessoa))

SELECT "PESSOA".id, "PESSOA".nome, "PESSOA".sobrenome 
FROM "PESSOA"


In [18]:
with Session(engine) as session:
    result = session.scalars(select(Pessoa))

    for pessoa in result:
        print(f'{pessoa.id} -> {pessoa.nome} {pessoa.sobrenome}')

2023-11-26 18:38:59,281 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-26 18:38:59,282 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id, "PESSOA".nome, "PESSOA".sobrenome 
FROM "PESSOA"
2023-11-26 18:38:59,283 INFO sqlalchemy.engine.Engine [generated in 0.00044s] ()
1 -> Jubileu None
2 -> José Pocai
2023-11-26 18:38:59,284 INFO sqlalchemy.engine.Engine ROLLBACK
